In [1]:
import tensorflow as tf

# session

###### old style

In [2]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

print(result)

sess.close()

42


###### new style1

In [3]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    # same as - tf.get_default_session().run(x.initializer)
    
    result = f.eval()
    # same as - tf.get_default_session().run(f)
''
print(result)

42


###### new style2

In [4]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
''
print(result)

42


###### where is the session?

> - sess = tf.Session()  이렇게 생성한 세션은 default session으로 자동 등록되지 않는다.
> - with 문 안에서는 with로 생성한 세션이 자동으로 default session으로 등록되고 with문이 끝날때 close된다.
> - sess = tf.InteractiveSession() 으로 세션을 생성할 경우 자동으로 default session으로 등록된다.

In [5]:
sess

In [6]:
print(tf.get_default_session())

None


In [7]:
with tf.Session() as sess:
    print(tf.get_default_session())

In [8]:
isess = tf.InteractiveSession()
print(tf.get_default_session())
isess.close()

###### executing variables

In [37]:
import tensorflow as tf

w = tf.Variable(initial_value=0, name='w')

with tf.Session() as sess:
    sess.run(w.initializer)
    
    # all these three commands are equivalent.
    _w = sess.run(w)
    # _w = tf.get_default_session().run(w)
    # _w = w.eval()
''
print(_w)

0


###### Saving and Restoring models

In [25]:
import tensorflow as tf

w = tf.Variable(initial_value=0, name='w')
k = w + 1
saver = tf.train.Saver()

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    _k = sess.run(k)
    save_path = saver.save(sess, '/tmp/my_model.ckpt')


In [26]:
with tf.Session() as sess:
    saver.restore(sess, '/tmp/my_model.ckpt')
    print(sess.graph)

INFO:tensorflow:Restoring parameters from /tmp/my_model.ckpt


# graphs

> - 변수를 생성할 때마다 default graph에 추가된다.
> - jupyter notebook에서 작업할 때 실수로 cell을 여러번 실행하면 중복(이름에 언더바-숫자가 붙은) 변수들이 마구 마구 생성된다.
> - default graph를 초기화 하려면 tf.reset_default_graph()를 실행한다.

In [9]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [11]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is graph

True

In [12]:
x2.graph is tf.get_default_graph()

False

# tensorboard

> - tensorboard --logdir tf_logs\
> - http://dongwankim-pc:6006/

In [2]:
import tensorflow as tf
from datetime import datetime

now = datetime.now().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

Something u wanna do

In [1]:
# mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

NameError: name 'tf' is not defined

# variable_scope and name_scope

> - https://github.com/ageron/handson-ml/blob/master/09_up_and_running_with_tensorflow.ipynb
> - handson-ml - Chapter9 - p248~

In [3]:
import tensorflow as tf
import numpy as np

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [4]:
reset_graph()

with tf.variable_scope("my_scope"):
    x0 = tf.get_variable("x", shape=(), initializer=tf.constant_initializer(0.))
    x1 = tf.Variable(0., name="x")
    x2 = tf.Variable(0., name="x")

with tf.variable_scope("my_scope", reuse=True):
    x3 = tf.get_variable("x")
    x4 = tf.Variable(0., name="x")

with tf.variable_scope("", default_name="", reuse=True):
    x5 = tf.get_variable("my_scope/x")

print("x0:", x0.op.name)
print("x1:", x1.op.name)
print("x2:", x2.op.name)
print("x3:", x3.op.name)
print("x4:", x4.op.name)
print("x5:", x5.op.name)
print(x0 is x3 and x3 is x5)

x0: my_scope/x
x1: my_scope/x_1
x2: my_scope/x_2
x3: my_scope/x
x4: my_scope_1/x
x5: my_scope/x
True


The first variable_scope() block first creates the shared variable x0, named my_scope/x. For all operations other than shared variables (including non-shared variables), the variable scope acts like a regular name scope, which is why the two variables x1 and x2 have a name with a prefix my_scope/. Note however that TensorFlow makes their names unique by adding an index: my_scope/x_1 and my_scope/x_2.

The second variable_scope() block reuses the shared variables in scope my_scope, which is why x0 is x3. Once again, for all operations other than shared variables it acts as a named scope, and since it's a separate block from the first one, the name of the scope is made unique by TensorFlow (my_scope_1) and thus the variable x4 is named my_scope_1/x.

The third block shows another way to get a handle on the shared variable my_scope/x by creating a variable_scope() at the root scope (whose name is an empty string), then calling get_variable() with the full name of the shared variable (i.e. "my_scope/x").

---

name_scope은 with로 열 때마다 기존에 동일 이름의 scope이 있는지 확인 후    
동일 이름이 없다면 지정한 이름으로 scope을 생성하고   
기존에 같은 이름이 있다면 지정한 이름 뒤에 언더바-숫자를 덛붙여 중복을 피한다.

variable_scope은 with문안에 어떤 방식으로 변수를 선언하느냐에 따라 그 동작이 다르다.   
get_variable로 변수를 생성/재사용하는 경우 지정한 이름을 고유하게 인식한다. 즉  언더바-숫자를 덛붙이는 일이 없다.(x3의 이름이 my_scope_1/x가 아니라 my_scope/x 이다.) 
반면 variable_scope안에서 Variable() 등올 변수를 생성하는 경우 name_scope과 동일하게 동작한다. 즉 기존에 동일한 이름으로 생성된 scope이 존재한다면 지정한 이름 뒤에 언더바-숫자를 덛붙여 중복을 피한다.(이 때문에 예제 코드에서 x4의 이름이 my_scope_1/x이다.)

또한 variable_scope에서 reuse 값을 어떻게 설정하느냐에 따라 동작이 달라진다.

- (default) reuse=False 인 경우 중복 없이 변수를 생성하는 것을 의도한다.
 - 기존에 동일 이름의 변수가 있으면 exception 발생
 - 기존에 동일 이름의 변수가 없다면 새로 생성
- reuse=True 인 경우 기존 변수를 재사용하는 것을 의도한다.
 - 기존에 동일 이름의 변수가 있으면 기존 것을 가져온다.
 - 기존에 동일 이름의 변수가 없다면 exception 발생

# placeholder, Variable, ... things to create variable node

###### > placeholder(
    dtype,
    shape=None,
    name=None
)

> https://www.tensorflow.org/api_docs/python/tf/placeholder

In [1]:
import tensorflow as tf

X = tf.placeholder(dtype=tf.float32, shape=(None, 3), name='X')
y = X + 3

with tf.Session() as sess:
    _y = sess.run(y, feed_dict={X: [[1, 2, 3]]})
    
print(_y)

[[ 4.  5.  6.]]


###### > Variable(
    initial_value=None,
    trainable=True,
    collections=None,
    validate_shape=True,
    caching_device=None,
    name=None,
    variable_def=None,
    dtype=None,
    expected_shape=None,
    import_scope=None,
    constraint=None
)

> https://www.tensorflow.org/api_docs/python/tf/Variable

In [19]:
import tensorflow as tf

w = tf.Variable(initial_value=0, name='w')

with tf.Session() as sess:
    sess.run(w.initializer)
    _w = sess.run(w)
''
print(_w)

0


###### > get_variable(
    name,
    shape=None,
    dtype=None,
    initializer=None,
    regularizer=None,
    trainable=True,
    collections=None,
    caching_device=None,
    partitioner=None,
    validate_shape=True,
    use_resource=None,
    custom_getter=None,
    constraint=None
)

> https://www.tensorflow.org/api_docs/python/tf/get_variable

In [26]:
tf.reset_default_graph()

with tf.variable_scope('weights'):
    s = tf.get_variable(name='s', shape=())
''

with tf.Session() as sess:
    sess.run(s.initializer)
    _s = sess.run(s)
''
print(_s)

0.113985


###### > constant(
    value,
    dtype=None,
    shape=None,
    name='Const',
    verify_shape=False
)

> https://www.tensorflow.org/api_docs/python/tf/constant

In [29]:
z = tf.constant(value=1, dtype=tf.float32, name='z')

with tf.Session() as sess:
    # sess.run(s.initializer)
    _z = sess.run(z)
''
print(_z)

1.0


# Commom modules

###### reset func in the notebook

In [17]:
import tensorflow as tf
import numpy as np

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
''
reset_graph()

# numpy utils

In [5]:
import numpy as np

Y = np.array([[5, 0, 2, 5, 2, 4, 2, 1, 1, 4]])

In [6]:
Y.reshape(-1)

array([5, 0, 2, 5, 2, 4, 2, 1, 1, 4])

In [7]:
np.eye(N=6)[Y.reshape(-1)]

array([[ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.]])

In [9]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [10]:
convert_to_one_hot(Y, 6)

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])